In [ ]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer
from nltk.corpus import words, stopwords

from google.colab import drive
drive.mount('/gdrive')

!python -m nltk.downloader all

pd.set_option('max_colwidth', 999)
pd.set_option('max_rows', 50)

In [8]:
def ___display_left_align___(df, n):
    col = 'text_list' if 'text_list' in df.columns else 'text'
    for i in range(n):
        print(df['label'][i], [df[col][i]])
    print()

In [3]:
PATH = "/gdrive/MyDrive/MLDM/Data Mining/Project/data"

df_train = pd.read_csv(PATH + "/DataFrame_train.csv")
df_test = pd.read_csv(PATH + "/DataFrame_test.csv")

df_train['text'] = df_train['text'].astype(str)
df_test['text'] = df_test['text'].astype(str)

df_train.head(10)

,id,label,text
0,1,0,"Showers continued throughout the week in\nthe Bahia cocoa zone, alleviating the drought since early\nJanuary and improving prospects for the coming temporao,\nalthough normal humidity levels have not been restored,\nComissaria Smith said in its weekly review.\n The dry period means the temporao will be late this year.\n Arrivals for the week ended February 22 were 155,221 bags\nof 60 kilos making a cumulative total for the season of 5.93\nmln against 5.81 at the same stage last year. Again it seems\nthat cocoa delivered earlier on consignment was included in the\narrivals figures.\n Comissaria Smith said there is still some doubt as to how\nmuch old crop cocoa is still available as harvesting has\npractically come to an end. With total Bahia crop estimates\naround 6.4 mln bags and sales standing at almost 6.2 mln there\nare a few hundred thousand bags still in the hands of farmers,\nmiddlemen, exporters and processors.\n There are doubts as to how much of this cocoa wo..."
1,2,0,"Standard Oil Co and BP North America\nInc said they plan to form a venture to manage the money market\nborrowing and investment activities of both companies.\n BP North America is a subsidiary of British Petroleum Co\nPlc &lt;BP>, which also owns a 55 pct interest in Standard Oil.\n The venture will be called BP/Standard Financial Trading\nand will be operated by Standard Oil under the oversight of a\njoint management committee.\n\n Reuter\n"
2,3,0,Texas Commerce Bancshares Inc's Texas\nCommerce Bank-Houston said it filed an application with the\nComptroller of the Currency in an effort to create the largest\nbanking network in Harris County.\n The bank said the network would link 31 banks having\n13.5 billion dlrs in assets and 7.5 billion dlrs in deposits.\n \n Reuter\n
3,4,0,"BankAmerica Corp is not under\npressure to act quickly on its proposed equity offering and\nwould do well to delay it because of the stock's recent poor\nperformance, banking analysts said.\n Some analysts said they have recommended BankAmerica delay\nits up to one-billion-dlr equity offering, which has yet to be\napproved by the Securities and Exchange Commission.\n BankAmerica stock fell this week, along with other banking\nissues, on the news that Brazil has suspended interest payments\non a large portion of its foreign debt.\n The stock traded around 12, down 1/8, this afternoon,\nafter falling to 11-1/2 earlier this week on the news.\n Banking analysts said that with the immediate threat of the\nFirst Interstate Bancorp &lt;I> takeover bid gone, BankAmerica is\nunder no pressure to sell the securities into a market that\nwill be nervous on bank stocks in the near term.\n BankAmerica filed the offer on January 26. It was seen as\none of the major factors leading..."
4,5,0,"The U.S. Agriculture Department\nreported the farmer-owned reserve national five-day average\nprice through February 25 as follows (Dlrs/Bu-Sorghum Cwt) -\n Natl Loan Release Call\n Avge Rate-X Level Price Price\n Wheat 2.55 2.40 IV 4.65 --\n V 4.65 --\n VI 4.45 --\n Corn 1.35 1.92 IV 3.15 3.15\n V 3.25 --\n X - 1986 Rates.\n\n Natl Loan Release Call\n Avge Rate-X Level Price Price\n Oats 1.24 0.99 V 1.65 -- \n Barley n.a. 1.56 IV 2.55 2.55\n V 2.65 -- \n Sorghum 2.34 3.25-Y IV 5.36 5.36\n V 5.54 -- \n Reserves I, II and III have matured. Level IV reflects\ngrain entered after Oct 6, 1981 for feedgrain and after July\n23, 1981 for wheat. Level V wheat/barle..."
5,6,0,"Argentine grain board figures show\ncrop registrations of grains, oilseeds and their products to\nFebruary 11, in thousands of tonnes, showing those for futurE\nshipments month, 1986/87 total and 1985/86 total to February\n12, 1986, in brackets:\n Bread wheat prev 1,655.8, Feb 872.0, March 164.6, total\n2,692.4 (4,161.0).\n Maize Mar 48.0, total 48.0 (nil).\n Sorghum nil (nil)\n Oilseed export registrations were:\n Sunflowerseed total 15.0 (7.9)\n Soybean May 20.0, total 20.0 (nil)\n The board also detailed export regi

# 

In [9]:
name = 'train'

N = 5

df_train = pd.read_csv(PATH + f"/DataFrame_{name}.csv")
df_train['text'] = df_train['text'].astype(str)
___display_left_align___(df_train, N)


# Replacing &lt; with <
print("===== Replacing &lt; with < =====")
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('&lt;', '<'))

# Removing "Reuter"
print("===== Removing 'Reuter' =====")
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('REUTER', ''))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('Reuter', ''))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('reuter', ''))

# Replacing "U.S."
print("===== Replacing 'U.S.' =====")
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('U.S.', 'us'))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('u.s.', 'us'))

___display_left_align___(df_train, N)


# Converting to lower case
print("===== Converting to lower case =====")
df_train['text'] = df_train['text'].apply(lambda txt: txt.lower())

# Removing double spaces and new lines
print("===== Removing double spaces and new lines =====")
df_train['text'] = df_train['text'].apply(lambda txt: " ".join(txt.split()))
___display_left_align___(df_train, N)




# Adding space at the beginning for replacing abreviations later
df_train['text'] = df_train['text'].apply(lambda txt: ' ' + txt)


# Removing , and . from numbers
print("===== Removing , and . from numbers =====")

def cleaning_numbers(txt):
    new_str = ""

    for i in range(len(txt)):
        if (txt[i] == ',') or (txt[i] == '.'):
            if (i > 0) and txt[i-1].isdigit(): # Check if previous char is digit
                if (i < len(txt)-1) and txt[i+1].isdigit(): # Check if next char is digit
                    continue
            
        new_str += txt[i]

    return new_str

df_train['text'] = df_train['text'].apply(lambda txt: cleaning_numbers(txt))
___display_left_align___(df_train, N)





# Replacing every number by '@'
print("===== Replacing every number by '@' =====")
def replace_numbers_to_at(txt):
    L = list(txt)
    pred = ''
    for i in range(len(L)):
        if L[i].isdigit():
            if pred != '@':
                L[i] = '@'
                pred = '@'
            else:
                L[i] = ''
        else:
            pred = ''

    return ''.join(L)


df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('billions', ''))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('billion', ''))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('millions', ''))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('million', ''))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('mln', ''))  # 'millions'

df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('@', ' '))
df_train['text'] = df_train['text'].apply(lambda txt: replace_numbers_to_at(txt))
___display_left_align___(df_train, N)



# Removing punctuation
# [' ', '"', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '>', '?', '@', '[', ']', '^']
print("===== Removing punctuation =====")
for char in [',', '.', ';', ':', '?', '!', '^', '/', '*', '-', '+', '~', '"',]:
    df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(char, ' '))

for char in ['<', '>', '(', ')', '[', ']', '{', '}', "'"]:
    df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(char, ''))


# dollar/euro/pounds to $
print("===== Converting dollar/euro/pounds to $ =====")
for word in ['dlrs', 'dlr', 'usd', '€', 'gbp', 'lbs', '£']:
    df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(word, '$'))

for word in [' dollars ', ' dollar ', ' cents ', ' cent ', ' cts ', ' euros ', ' euro ', ' eurs ', ' eur ', ' pounds ', ' pound ']:
    df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(word, ' $ '))


___display_left_align___(df_train, N)


# Abbreviations to full word
print("===== Abbreviations to full word =====")
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('pct', '%'))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('qtly', 'quality'))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('avg', 'average'))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('natl', 'national'))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' shrs ', ' share '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' shr ', ' share '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('qtlry', 'quarterly'))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' div ', ' dividend '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('mths', 'month'))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace('mth', 'month'))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' yr ', ' year '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' revs ', ' revenue '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' rev ', ' revenue '))


df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' jan ', ' january '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' feb ', ' february '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' mar ', ' march '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' apr ', ' april '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' may ', ' may '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' jun ', ' june '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' jul ', ' july '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' aug ', ' august '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' sep ', ' september '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' oct ', ' october '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' nov ', ' november '))
df_train['text'] = df_train['text'].apply(lambda txt: txt.replace(' dec ', ' december '))



# Removing double spaces and new lines
print("===== Removing double spaces and new lines =====")
df_train['text'] = df_train['text'].apply(lambda txt: " ".join(txt.split()))
___display_left_align___(df_train, N)





# Word Tokenization
print("===== Tokenization =====")
df_train['text_list'] = df_train['text'].apply(lambda txt: nltk.word_tokenize(txt))
___display_left_align___(df_train, N)



# Removing stop words
print("===== Removing stop words =====")

def removing_stopwords(L_txt):
    L = []
    for word in L_txt:
        if word not in S_stopwords:
            L.append(word)
    return L

# print(stopwords.words('english'))
# removed : 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'up', 'down', 'more', 'over', 'under'

S_stopwords = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "youre", "youve", "youll", "youd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "shes", \
               'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "thatll", 'these', 'those', 'am', 'is', 'are', \
               'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', \
               'for', 'with', 'about', 'to', 'from', 'in', 'out', 'on', 'off', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'most', 'other', \
               'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "dont", 'should', "shouldve", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',\
               'ain', 'aren', "arent", 'couldn', "couldnt", 'didn', "didnt", 'doesn', "doesnt", 'hadn', "hadnt", 'hasn', "hasnt", 'haven', "havent", 'isn', "isnt", 'ma', 'mightn', "mightnt", 'mustn', "mustnt", \
               'needn', "neednt", 'shan', "shant", 'shouldn', "shouldnt", 'wasn', "wasnt", 'weren', "werent", 'won', "wont", 'wouldn', "wouldnt", 'nil', 'per', \
               'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', \
               'i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x'}

df_train['text_list'] = df_train['text_list'].apply(lambda L_txt: removing_stopwords(L_txt))
___display_left_align___(df_train, N)



# Word Lemmatization
print("===== Lemmatization =====")
def lemmatization(L_txt):
    L = []
    for word in L_txt:
        if len(word) <= 3 or word in ['pass', 'mars']:
            if word in S_english_words or word in ['$', '%', '@']:
                L.append(word)
        else:
            lem_word = lemmatizer.lemmatize(word)
            if lem_word in S_english_words:
                L.append(lem_word)
    return L


L_english_words = [word.lower() for word in words.words()]
S_english_words = set(L_english_words)

lemmatizer = WordNetLemmatizer()
df_train['text_list'] = df_train['text_list'].apply(lambda L_txt: lemmatization(L_txt))
___display_left_align___(df_train, N)



df_train = df_train.drop('text', axis=1)
df_train['text'] = df_train['text_list']
df_train = df_train.drop('text_list', axis=1)

0 ['Showers continued throughout the week in\nthe Bahia cocoa zone, alleviating the drought since early\nJanuary and improving prospects for the coming temporao,\nalthough normal humidity levels have not been restored,\nComissaria Smith said in its weekly review.\n    The dry period means the temporao will be late this year.\n    Arrivals for the week ended February 22 were 155,221 bags\nof 60 kilos making a cumulative total for the season of 5.93\nmln against 5.81 at the same stage last year. Again it seems\nthat cocoa delivered earlier on consignment was included in the\narrivals figures.\n    Comissaria Smith said there is still some doubt as to how\nmuch old crop cocoa is still available as harvesting has\npractically come to an end. With total Bahia crop estimates\naround 6.4 mln bags and sales standing at almost 6.2 mln there\nare a few hundred thousand bags still in the hands of farmers,\nmiddlemen, exporters and processors.\n    There are doubts as to how much of this cocoa wou